# COSC 411: Artificial Intelligence

Instructor: Dr. Shuangquan (Peter) Wang

Email: spwang@salisbury.edu

Department of Computer Science, Salisbury University


# Module 3_ML algorithms and application

## 5. Ensemble Methods



**Contents of this note refer to**
1) https://colab.research.google.com/github/Eldave93/Seizure-Detection-Tutorials/blob/master/05.%20Ensemble%20Learning.ipynb#:~:text=Ensemble%20methods%20aim%20to%20improve,general%20methods%2C%20averaging%20and%20boosting
2) Data Science Complete Tutorial. https://github.com/edyoda/data-science-complete-tutorial/blob/master/16.%20Ensemble%20Methods.ipynb
3) Python toturial: https://docs.python.org/3/tutorial/

**<font color=red>All rights reserved. Dissemination or sale of any part of this note is NOT permitted.</font>**

# Ensemble Methods
* Objective of ensemble methods is to combine the predictions of serveral base estimators ( Linear Regression, Decisison Tree, etc. ) to create a combined effect or more generalized model.
* Two types of Ensemble Method
  - Averaging Method : Build several estimators independently & average their predictions. Examples are RandomForest etc.
  - Boosting Method : Base estimators are built sequentially using weighted version of data, i.e. fitting models with data that were mis-classified. Examples are AdaBoost
  
<img src="https://cdn-images-1.medium.com/max/1000/1*PaXJ8HCYE9r2MgiZ32TQ2A.png">

## Bagging
A bagging classifier is an ensemble of base classifiers, each fit on random subsets of a dataset. Their predictions are then pooled or aggregated to form a final prediction. This reduces variance of an estimator and can be a simple way to reduce overfitting. They work best with complex models as opposed to boosting, which work best with weak models<sup>1</sup>.

Specifically, bagging is when sampling is produced with replacement<sup>2</sup>, and without replacement being called pasting<sup>3</sup>. Therefore both bagging and pasting allow training to be sampled several times across multipule predictors, with bagging only allowing several samples for the same predictor <sup>4</sup>.

We can do this with any classifier so lets start with a support vector machine.

**NOTE**
- If we wanted to use pasting we would just set *bootstrap=False*.

---

1. https://scikit-learn.org/stable/modules/ensemble.html
2. Breiman, L. (1996). Bagging predictors. Machine learning, 24(2), 123-140.
3. Breiman, L. (1999). Pasting small votes for classification in large databases and on-line. Machine learning, 36(1-2), 85-103.
4. Géron, A. (2017). Hands-on machine learning with Scikit-Learn and TensorFlow: concepts, tools, and techniques to build intelligent systems. " O'Reilly Media, Inc.".

In [1]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report

pipe_svc = Pipeline([('scl', StandardScaler()),
                     ('pca', PCA(n_components=0.8, random_state = 0)),
                     ('clf', SVC(kernel='rbf', random_state=0))])

bag = BaggingClassifier(base_estimator=pipe_svc,
                        n_estimators=10,
                        max_samples=0.5,
                        max_features=0.5,
                        bootstrap=True,
                        bootstrap_features=True,
                        oob_score=True,
                        warm_start=False,
                        n_jobs=-1,
                        random_state=0)



In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

dataset = load_iris()
#df = pd.DataFrame(dataset.data, columns=data.feature_names)
X = dataset['data']
y = dataset['target']

X_train, X_test, y_train, y_test = \
train_test_split(X, y, test_size=0.3, random_state=0)

bag.fit(X_train, y_train)

C:\Users\George\AppData\Local\anaconda3\Lib\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


BaggingClassifier(base_estimator=Pipeline(steps=[('scl', StandardScaler()),
                                                 ('pca',
                                                  PCA(n_components=0.8,
                                                      random_state=0)),
                                                 ('clf', SVC(random_state=0))]),
                  bootstrap_features=True, max_features=0.5, max_samples=0.5,
                  n_jobs=-1, oob_score=True, random_state=0)

In [3]:
y_pred = bag.predict(X_test)

display(pd.DataFrame(classification_report(y_test, y_pred , output_dict =True)))

,0,1,2,accuracy,macro avg,weighted avg
precision,1.0,0.900000,1.000000,0.955556,0.966667,0.960000
recall,1.0,1.000000,0.818182,0.955556,0.939394,0.955556
f1-score,1.0,0.947368,0.900000,0.955556,0.949123,0.954503
support,16.0,18.000000,11.000000,0.955556,45.000000,45.000000


As we can see, performance is okay but recall is particularly poor. It is likely the model is not complex enough or models in the ensemble are too similar to each other (we'll look at solving this soon).

An additional way we can get a performance metric on a validation set is to ensure we use `oob_score = True`

With bagging by default only trains on a sample of the training data, leaving a set of training data sampled as out-of-bag (oob) instances. Since they are not seen during training, we can evalute on them without a separate validation using the oob_score.

It gets an accuracy of about 0.94 on the test/validation set, so it pretty close to what we did get above.

In [16]:
bag.oob_score_

0.9428571428571428

## RandomForest
* Limitations of decison tree is that it overfits & shows high variance.
* RandomForest is an averaging ensemble method whose prediction is function of prediction of 'n' decision trees.

<img src="https://www.researchgate.net/profile/Stavros_Dimitriadis/publication/324517994/figure/fig1/AS:615965951799303@1523869135381/Classification-process-based-on-the-Random-Forest-algorithm-2.png">

##### Algorithm
* Data consist of R rows & M features.
* Sample of training data is taken.
* Random set of features are selected.
* As many as configured number of trees are created using above two steps.
* Final prediction in case of classification is majority prediction.
* Final prediction in case of regression is mean/median of individual tree prediction

##### Comparing Decision Tree & Random Forest for MNIST data

In [5]:
from sklearn.datasets import load_digits
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [6]:
digits = load_digits()

In [7]:
X = digits.data
y = digits.target

In [8]:
trainX, testX, trainY, testY = train_test_split(X,y)

In [9]:
dt = DecisionTreeClassifier()

In [10]:
dt.fit(trainX,trainY)

DecisionTreeClassifier()

In [11]:
dt.score(testX,testY)

0.8444444444444444

In [12]:
rf = RandomForestClassifier()

In [13]:
rf.fit(trainX,trainY)

RandomForestClassifier()

In [14]:
rf.score(testX,testY)

0.9733333333333334

##### Important Hyper-parameters
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
* n_estimators : number of trees to be configured, larger is better but compute cost.
* max_features : maximum number of features to be considered for splitting the node. For classification this equals to sqrt(n_features). And, for regression max_features = n_features.
* n_jobs : Configure as -1 so that we can make use of all processors.

#### Advantages
* Minimal data cleaning or dealing with missing values required.
* Works well with high dimensional datasets
* Minimizes variance even for low variance models
* RandomForest can tell importance of features. We can find important features & use them in model training

In [15]:
rf.feature_importances_

array([0.00000000e+00, 2.92165259e-03, 2.11962371e-02, 1.02377314e-02,
       1.02049335e-02, 2.36042834e-02, 9.68052909e-03, 7.35899090e-04,
       5.09084220e-05, 9.24183712e-03, 2.84201100e-02, 7.01493872e-03,
       1.46134169e-02, 3.14818860e-02, 6.21942877e-03, 7.92124214e-04,
       1.40916527e-04, 5.73589177e-03, 2.34180627e-02, 2.43347860e-02,
       3.07849963e-02, 4.89081647e-02, 9.02106832e-03, 4.01984040e-04,
       0.00000000e+00, 1.63847063e-02, 3.41280851e-02, 2.71898284e-02,
       2.87485702e-02, 2.17326712e-02, 2.91090883e-02, 5.01181679e-05,
       0.00000000e+00, 3.40650606e-02, 2.71283506e-02, 1.50354254e-02,
       3.94457176e-02, 1.86173622e-02, 2.89420968e-02, 0.00000000e+00,
       2.85485148e-05, 1.44271096e-02, 3.83420910e-02, 4.04454422e-02,
       2.26319774e-02, 1.65774439e-02, 2.26870013e-02, 1.79756021e-04,
       3.60906527e-05, 3.24799972e-03, 1.66641574e-02, 2.39763847e-02,
       1.29254912e-02, 2.06374767e-02, 2.33940230e-02, 1.64014005e-03,
      

## AdaBoost
* Boosting in general is about building a model from the training data, then creating a second model that attempts to correct the errors from the first model. Models are added until the training set is predicted perfectly or a maximum number of models are added.

##### Algorithm
* Core concept of adaboost is to fit weak learners ( like decision tree ) sequantially on repeatedly modifying data.
* Initially, each data is assigned equal weights.
* A base estimator is fitted with this data.
* Weights of misclassified data are increased & weights of correctly classified data is decreased. 
* Repeat the above two steps till all data are correctly classified or max number of iterations configured.
* Making Prediction : The predictions from all of them are then combined through a weighted majority vote (or sum) to produce the final prediction.

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
ab = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=8),n_estimators=600)

In [ ]:
ab.fit(trainX,trainY)

In [ ]:
ab.score(testX,testY)

In [ ]:
ab = AdaBoostClassifier(base_estimator=RandomForestClassifier(n_estimators=20),n_estimators=600)

In [ ]:
ab.fit(trainX,trainY)

In [ ]:
ab.score(testX,testY)